In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
from fastai.vision import *

In [ ]:
data_path = Path('/kaggle/input/intel-image-classification/')

In [ ]:
data = ImageList.from_folder(data_path)\
.split_by_folder(train='seg_train', valid='seg_test')\
.label_from_folder()\
.add_test_folder(test_folder='seg_pred')\
.transform(get_transforms(), size=224)\
.databunch()\
.normalize(imagenet_stats)

In [ ]:
print(data.classes)
print(data.c)

In [ ]:
data.show_batch(rows=3, figsize=(7,7))

In [ ]:
from os.path import expanduser, join, exists
from os import makedirs
cache_dir = expanduser(join('~', '.cache'))
if not exists(cache_dir):
	makedirs(cache_dir)
torch_dir = join(cache_dir, 'torch')
if not exists(torch_dir):
	makedirs(torch_dir)
checkpoints_dir = join(torch_dir, 'checkpoints')
if not exists(checkpoints_dir):
	makedirs(checkpoints_dir)
print(checkpoints_dir)

!cp ../input/resnet34/resnet34.pth /tmp/.cache/torch/checkpoints/resnet34-333f7ec4.pth
!cp ../input/resnet50/resnet50.pth /tmp/.cache/torch/checkpoints/resnet50-19c8e357.pth

In [ ]:
learner_34 = cnn_learner(data, models.resnet34,metrics=[accuracy, error_rate])

In [ ]:
learner_34.model_dir='/kaggle/working/'

In [ ]:
learner_34.lr_find()
learner_34.recorder.plot()

In [ ]:
# learner_34 = cnn_learner(data, models.resnet34,metrics=[accuracy, error_rate])
# learner_34.fit_one_cycle(6, 8e-3)

In [ ]:
# learner_34 = cnn_learner(data, models.resnet34,metrics=[accuracy, error_rate])
# train only later layers, as the current image set is specific. Not all resnet34 layers are needed here for classification
learner_34.freeze()
learner_34.fit_one_cycle(6, 8e-3)

In [ ]:
learner_34.save('learner_34_v1')

In [ ]:
# learner_34.unfreeze()
# learner_34.fit_one_cycle(2, max_lr=slice(1e-3, 3e-2))
# learner_34.fit_one_cycle(2)

In [ ]:
# learner_34.export('/kaggle/working/export.pkl')

In [ ]:
# pred_list = ImageList.from_folder(data_path/'seg_pred')
# learner34_inference = load_learner('/kaggle/working/', test=pred_list)

In [ ]:
# preds, y = learner34_inference.get_preds(ds_type=DatasetType.Test)

In [ ]:
# print(len(preds))
# print(len(pred_list))

In [ ]:
interp = ClassificationInterpretation.from_learner(learner_34)

In [ ]:
interp.plot_top_losses(9, figsize=(8,8))

In [ ]:
interp.plot_confusion_matrix(figsize=(10,10), dpi=75)

In [ ]:
interp.most_confused(min_val=2)

**Try with a RESNET50 model**

In [ ]:
data = ImageList.from_folder(data_path)\
.split_by_folder(train='seg_train', valid='seg_test')\
.label_from_folder()\
.add_test_folder(test_folder='seg_pred')\
.transform(get_transforms(), size=224)\
.databunch()\
.normalize(imagenet_stats)

In [ ]:
learner_50 = cnn_learner(data, models.resnet50, metrics=[accuracy, error_rate])

In [ ]:
learner_50.model_dir='/kaggle/working/'

In [ ]:
learner_50.lr_find()
learner_50.recorder.plot()

In [ ]:
learner_50.freeze()
learner_50.fit_one_cycle(5, 8e-3)

In [ ]:
learner_50.save('learner_50_v1')

In [ ]:
interp_50 = ClassificationInterpretation.from_learner(learner_50)

In [ ]:
interp_50.plot_top_losses(9, figsize=(8,8))

In [ ]:
interp_50.plot_confusion_matrix(figsize=(12,12), dpi=75)

In [ ]:
interp_50.most_confused(min_val=2)

In [ ]:
learner_50.fit_one_cycle(2)